In [1]:
from prody import *
import numpy as np
import matplotlib.pyplot as plt
import os
import fnmatch

In [2]:
def new_dihedral(p):
    """Praxeolitic formula
    1 sqrt, 1 cross product
    https://stackoverflow.com/questions/20305272/dihedral-torsion-angle-from-four-points-in-cartesian-coordinates-in-python
    """
    p0 = p[0]
    p1 = p[1]
    p2 = p[2]
    p3 = p[3]

    b0 = p1 - p0
    b1 = p2 - p1
    b2 = p3 - p2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    b1 /= np.linalg.norm(b1)

    # vector rejections
    # v = projection of b0 onto plane perpendicular to b1
    #   = b0 minus component that aligns with b1
    # w = projection of b2 onto plane perpendicular to b1
    #   = b2 minus component that aligns with b1
    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

In [3]:


def process_files(pdb_file, dcd_file):
    pdb = parsePDB(pdb_file)
    dcd = parseDCD(dcd_file)

    dcd.setAtoms(pdb)
    dcd.setCoords(pdb)
    dcd.superpose()

    distances = []
    dihedrals = []

    for i in range(len(dcd)):
        
        selection_1 = pdb.select('calpha and resnum 5to68')
        dcd.setAtoms(selection_1)
        coord_1 = calcCenter(dcd[i].getCoords(), weights=dcd[i].getAtoms().getMasses())
        
        selection_2 = pdb.select('calpha and resnum 92to147')
        dcd.setAtoms(selection_2)
        coord_4 = calcCenter(dcd[i].getCoords(), weights=dcd[i].getAtoms().getMasses())
        
        selection_3 = pdb.select('calpha and resnum 69')
        dcd.setAtoms(selection_3)
        coord_2 = dcd[i].getCoords()[0]
        
        selection_4 = pdb.select('calpha and resnum 91')
        dcd.setAtoms(selection_4)
        coord_3 = dcd[i].getCoords()[0]
        
        dihedral = new_dihedral([coord_1, coord_2, coord_3, coord_4]) + 180
        
        # Adjust dihedral angle as specified
        if dihedral > 180:
            dihedral -= 360
        
        distance = calcDistance(coord_2, coord_3)
        
        # Exclude entries with dihedral > 175 or < -175 and distance within the specified range
        if not ((dihedral > 175 or dihedral < -175) and (distance < 10.75 or distance > 49.75)):
            dihedrals.append(dihedral)
            distances.append(distance)
    
    # Save the results
    base_name = os.path.splitext(os.path.basename(pdb_file))[0]
    np.savetxt(f"{base_name}_distance.dat", distances)
    np.savetxt(f"{base_name}_dihedral.dat", dihedrals)

# Get list of PDB and DCD files
pdb_files = fnmatch.filter(os.listdir(), "*.pdb")
dcd_files = fnmatch.filter(os.listdir(), "*.dcd")

# Process each pair of PDB and DCD files
for pdb_file in pdb_files:
    dcd_file = pdb_file.replace(".pdb", ".dcd")
    if dcd_file in dcd_files:
        process_files(pdb_file, dcd_file)


@> 2184 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> DCD file contains 1001 coordinate sets for 2184 atoms.
@> DCD file was parsed in 0.03 seconds.
@> 25.10 MB parsed at input rate 784.67 MB/s.
@> 1001 coordinate sets parsed at input rate 31298 frame/s.
@> Superposition completed in 1.03 seconds.
